In [ ]:
%pip install tensorflow

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# --- Parte 1: Cargar y preparar los datos históricos (igual que el original) ---
df = pd.read_excel(
    "C:/Users/mferrera/Documents/Ventaspipsagt.xlsx",
    sheet_name="Hoja1",
    skiprows=2
)

df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Fecha_Anio_Mes'] = df['Fecha'].dt.strftime('%Y-%m')

promedio_mensual = df.groupby('Fecha_Anio_Mes').agg(
    total_ventas=('TotalVenta', 'sum'),
    total_costos=('CostoTotal', 'sum'),
    total_ton=('Total TON vendidas', 'sum')
).reset_index()

promedio_mensual['Margen'] = (promedio_mensual['total_ventas'] - promedio_mensual['total_costos']) / promedio_mensual['total_ventas']

# --- Parte 2: Predicción de Margen Futuro con TensorFlow (LSTM) ---
print("\n--- Realizando predicción con un modelo LSTM de TensorFlow ---")

# 1. Preparar los datos para la LSTM
# Extraemos la serie de tiempo que nos interesa (Margen)
data_margen = promedio_mensual['Margen'].values.reshape(-1, 1)

# Normalizamos los datos para que estén en el rango de [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data_margen)

# 2. Función para crear el dataset de la LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# Creamos el dataset. 'look_back' es el número de meses que se usan para predecir el siguiente.
# En este caso, usaremos el margen del mes anterior para predecir el actual.
look_back = 1
X, y = create_dataset(data_scaled, look_back)

# Reformateamos los datos para que la LSTM los pueda procesar: [muestras, pasos_de_tiempo, características]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# 3. Crear y entrenar el modelo LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=100, batch_size=1, verbose=0) # Entrenamos el modelo

# 4. Realizar las predicciones para los próximos 3 meses
ult_dato = data_scaled[-look_back:]
predicciones_lstm = []
for _ in range(3):
    input_pred = ult_dato.reshape((1, look_back, 1))
    prediccion_scaled = model.predict(input_pred, verbose=0)
    predicciones_lstm.append(prediccion_scaled[0][0])
    ult_dato = np.append(ult_dato[1:], prediccion_scaled, axis=0)

# 5. Revertir la normalización de las predicciones
predicciones_margen = scaler.inverse_transform(np.array(predicciones_lstm).reshape(-1, 1))

df_predicciones = pd.DataFrame({
    'Margen': predicciones_margen.flatten()
})
last_date = pd.to_datetime(promedio_mensual['Fecha_Anio_Mes']).max()
df_predicciones['Fecha_Anio_Mes'] = [(last_date + pd.DateOffset(months=i)).strftime('%Y-%m') for i in range(1, 4)]

print("\nPredicciones de Margen para los próximos 3 meses (con LSTM):")
print(df_predicciones)

# --- Parte 3: Crear el gráfico combinado (doble eje) (igual que el original) ---
fig, ax1 = plt.subplots(figsize=(12, 7))

ax1.set_xlabel('Mes', fontsize=12)
ax1.set_ylabel('Total TON Vendidas', color='r', fontsize=12)
ax1.bar(promedio_mensual['Fecha_Anio_Mes'], promedio_mensual['total_ton'],
        color='r', alpha=0.5, label='Total TON Vendidas')
ax1.tick_params(axis='y', labelcolor='r')
plt.xticks(rotation=45, ha='right')

ax2 = ax1.twinx()
ax2.set_ylabel('Margen', color='b', fontsize=12)

ax2.plot(promedio_mensual['Fecha_Anio_Mes'], promedio_mensual['Margen'],
         marker='o', linestyle='-', color='b', label='Margen Histórico')

ax2.plot(df_predicciones['Fecha_Anio_Mes'], df_predicciones['Margen'],
         marker='x', linestyle='--', color='b', label='Predicción de Margen (LSTM)')

ax2.tick_params(axis='y', labelcolor='b')

indice_transicion = len(promedio_mensual) - 1
plt.axvline(x=promedio_mensual['Fecha_Anio_Mes'][indice_transicion], color='k', linestyle=':', linewidth=1.5)

plt.title('Análisis de Total de TONs Vendidas y Margen con Predicción (LSTM)', fontsize=16)

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper right')

plt.tight_layout()
#plt.show()
plt.savefig('analisisPIPSAGT_LSTM.jpg')

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.